In [102]:
import pandas as pd
from collections import Counter
import re
import math
df = pd.read_csv("./Data/201901_DBSCAN_CV.csv", encoding='utf-8-sig')

In [103]:
df = df.sort_values(by='labels', ascending=True)
df.tail()

,N_list,category,date,headline,img,press,text,topics,url,labels,cnt
345,"[['일본군', '위안부', '피해자', '김복동', '사진', '할머니', '별세...",국제,2019/01/29,유엔서 ‘위안부 피해’ 증언 김복동 할머니 별세,[],국민일보,일본군 위안부 피해자인 김복동(사진) 할머니가 28일 별세했다. 향년 93세.김 할...,"위안부, 피해, 김, 별세, 일본군, 피해자, 세, 암, 할머니",http://news.kmib.co.kr/article/view.asp?arcid=...,29,9
346,"[['위안부', '피해자', '인권', '운동', '기부', '김복동', '할머니'...","국제,사회",2019/01/28,'위안부 피해자' 김복동 할머니 별세… 생존자 23명으로,['//img.segye.com/content/image/2019/01/28/201...,세계일보,위안부 피해자로서 인권운동과 기부에 앞장서 온 김복동 할머니가 28일 별세했다. 향...,"위안부, 김, 피해, 피해자, 김복동, 기부, 할머니, 일본군, 딸, 별세",http://www.segye.com/content/html/2019/01/28/2...,29,9
347,"[['일본군', '위안부', '피해자', '김복동', '할머니', '별세'], ['...",국제,2019/01/28,‘위안부 피해’ 김복동 할머니마저 떠났다,['http://img.hani.co.kr/imgdb/resize/2019/0128...,한겨레,일본군 ‘위안부’ 피해자 김복동 할머니가 별세했다. 향년 93. 암 투병 중이던 김...,"할머니, 피해, 별세, 위안부, 일본군, 기금, 기억, 학교, 김, 재일",http://www.hani.co.kr/arti/society/society_gen...,29,9
348,"[['일본군', '위안부', '피해자', '이모', '할머니', '오전', '반',...",국제,2019/01/29,"위안부 피해 할머니 또 별세, 생존자 24명으로 줄어",['https://dimg.donga.com/a/258/230/90/2/wps/NE...,동아일보,일본군 위안부 피해자 이모 할머니가 28일 오전 7시 반경 노환으로 별세했다. 향년...,"피해자, 별세, 일본, 할머니, 세, 뒤, 위안부",http://news.donga.com/3/all/20190129/93900903/1,29,9
340,"[['일본', '아베', '총리', '진심', '사죄', '일본군', '위안부', ...",국제,2019/01/29,‘아이 캔 스피크’ 주인공 김복동 할머니 별세…이루지 못한 생전 소원은?,['http://image.kmib.co.kr/online_image/2019/01...,국민일보,“죽기 전에 일본 아베 총리의 진심 어린 사죄를 받고 싶다”고 했던 일본군 위안부 ...,"말씀, 위안부, 피해자, 할머니, 끝, 일본군, 세계, 피해, 기부, 저희",http://news.kmib.co.kr/article/view.asp?arcid=...,29,9


In [104]:
def npmi(a, b, n, z):
    x = math.log((n*z)/(a*b))
    y = math.log(n/z)
    return x / y

In [109]:
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        context += arr

    #2 PMI 적용
    for line in doc:
        com += list(zip(line, line[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+" "+key[1] for key, item in result if item >= 0.91]

    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = ' '.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)

In [134]:
df['N_list'] = docs
df.tail()

,N_list,category,date,headline,img,press,text,topics,url,labels,cnt
345,"[[일본군, 위안부, 피해자, 김복동, 사진, 할머니, 별세], [향, 세, 김, ...",국제,2019/01/29,유엔서 ‘위안부 피해’ 증언 김복동 할머니 별세,[],국민일보,일본군 위안부 피해자인 김복동(사진) 할머니가 28일 별세했다. 향년 93세.김 할...,"위안부, 피해, 김, 별세, 일본군, 피해자, 세, 암, 할머니",http://news.kmib.co.kr/article/view.asp?arcid=...,29,9
346,"[[위안부 피해자, 인권, 운동, 기부, 김복동, 할머니, 별세], [향, 세, 정...","국제,사회",2019/01/28,'위안부 피해자' 김복동 할머니 별세… 생존자 23명으로,['//img.segye.com/content/image/2019/01/28/201...,세계일보,위안부 피해자로서 인권운동과 기부에 앞장서 온 김복동 할머니가 28일 별세했다. 향...,"위안부, 김, 피해, 피해자, 김복동, 기부, 할머니, 일본군, 딸, 별세",http://www.segye.com/content/html/2019/01/28/2...,29,9
347,"[[일본군, 위안부, 피해자, 김복동, 할머니, 별세], [향, 암, 투병, 김, ...",국제,2019/01/28,‘위안부 피해’ 김복동 할머니마저 떠났다,['http://img.hani.co.kr/imgdb/resize/2019/0128...,한겨레,일본군 ‘위안부’ 피해자 김복동 할머니가 별세했다. 향년 93. 암 투병 중이던 김...,"할머니, 피해, 별세, 위안부, 일본군, 기금, 기억, 학교, 김, 재일",http://www.hani.co.kr/arti/society/society_gen...,29,9
348,"[[일본군, 위안부, 피해자, 이모, 할머니, 오전, 반, 노환, 별세], [향, ...",국제,2019/01/29,"위안부 피해 할머니 또 별세, 생존자 24명으로 줄어",['https://dimg.donga.com/a/258/230/90/2/wps/NE...,동아일보,일본군 위안부 피해자 이모 할머니가 28일 오전 7시 반경 노환으로 별세했다. 향년...,"피해자, 별세, 일본, 할머니, 세, 뒤, 위안부",http://news.donga.com/3/all/20190129/93900903/1,29,9
340,"[[일본, 아베, 총리, 진심, 사죄, 일본군, 위안부, 피해자, 김복동, 할머니,...",국제,2019/01/29,‘아이 캔 스피크’ 주인공 김복동 할머니 별세…이루지 못한 생전 소원은?,['http://image.kmib.co.kr/online_image/2019/01...,국민일보,“죽기 전에 일본 아베 총리의 진심 어린 사죄를 받고 싶다”고 했던 일본군 위안부 ...,"말씀, 위안부, 피해자, 할머니, 끝, 일본군, 세계, 피해, 기부, 저희",http://news.kmib.co.kr/article/view.asp?arcid=...,29,9


In [141]:
import pandas as pd
from gensim.models import word2vec
from gensim.models import Word2Vec

word_list = []
for i in range(df['labels'].max()+1):
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        for line in article:
            doc += line
        word_list.append(doc)
# topics = df['topics']
# topics = topics.dropna()
# for topic in topics:
#     word = topic.split(', ')
#     for w in word:
#         if(len(w) <= 1):
#             word.remove(w)
#     word_list.append(word)
# # model = Word2Vec.load('C:/Users/ghdtk/Project_test/ko.bin')
# # model.wv.save_word2vec_format('C:/Users/ghdtk/Project_test/ko.bin.gz', binary = False)
my_model = Word2Vec(word_list, window = 100, size = 200, min_count = 1, sg = 1)


# my_model.intersect_word2vec_format('C:/Users/ghdtk/Project_test/ko.bin.gz', binary = False)

word_vectors = my_model.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]

# print(word_vectors.similarity(w1='정부', w2='대통령'))
print(word_vectors.most_similar("문재인"))


454
